# Trabalho final de Sistemas Baseados em Conhecimento
### Tema: Geração de texto com Markov
### Alunos: Caio Lucas da Silva Chacon (20200025769) e Jonas Gabriel Leite de Araújo (20200007608)

### Dado o início do texto em português, gerar automaticamente algo que faça sentido usando cadeias de Markov

In [81]:
from scipy.sparse import dok_matrix
import numpy as np
from random import random 

# Leitura do arquivo e processamento textual para melhor análise

def read_text(filename):
    """"
    Função que lê um arquivo de texto e retorna o conteúdo em minúsculas
    """

    with open(filename, 'r', encoding="utf8") as file:
        content = file.read().lower()
        
        content = content.replace('\n', ' ')
        content = content.replace(',', ' ')
        content = content.replace('.', ' ')
        content = content.replace(';', ' ')
        content = content.replace(':', ' ')
        content = content.replace('"', ' ')
        content = content.replace("'", ' ')
        content = content.replace('“', ' ')
        content = content.replace('”', ' ')
        content = content.replace('—', ' ')
        content = content.replace('-', ' ')
        content = content.replace('_', ' ')
        content = content.replace('…', ' ')
        content = content.replace('\t', ' ')
        content = content.replace('  ', ' ')

    return content

content = read_text('domCasmurro.txt')
print(content)


dom casmurro capítulo primeiro do título uma noite destas vindo da cidade para o engenho novo encontrei no trem da central um rapaz aqui do bairro que eu conheço de vista e de chapéu cumprimentou me sentou se ao pé de mim falou da lua e dos ministros e acabou recitando me versos a viagem era curta e os versos pode ser que não fossem inteiramente maus sucedeu porém que como eu estava cansado fechei os olhos três ou quatro vezes tanto bastou para que ele interrompesse a leitura e metesse os versos no bolso  continue disse eu acordando  já acabei murmurou ele  são muito bonitos vi lhe fazer um gesto para tirá los outra vez do bolso mas não passou do gesto estava amuado no dia seguinte entrou a dizer de mim nomes feios e acabou alcunhando me dom casmurro os vizinhos que não gostam dos meus hábitos reclusos e calados deram curso à alcunha que afinal pegou nem por isso me zanguei contei a anedota aos amigos da cidade e eles por graça chamam me assim alguns em bilhetes  dom casmurro domingo v

In [82]:
def get_words(corpus):
    """
    Retorna todas as palavras
    """
    corpus_words = corpus.split(' ')
    corpus_words= [word for word in corpus_words if word != '']
    corpus_words # [...'a', 'wyvern', ',', 'two', 'of', 'the', 'thousand'...]
    return corpus_words # 2185920

def get_distinct_words_count(corpus_words):
    """
    Função que retorna a quantidade de palavras distintas e um dicionário com o índice de cada palavra
    """

    distinct_words = list(set(corpus_words))
    word_idx_dict = {word: i for i, word in enumerate(distinct_words)}
    distinct_words_count = len(list(set(corpus_words)))
    return distinct_words_count, word_idx_dict # 32663

In [83]:
corpus_words = get_words(content)
distinct_words = list(set(corpus_words))

distinct_words_count = get_distinct_words_count(corpus_words)
word_idx_dict = distinct_words_count[1]
distinct_words_count = distinct_words_count[0]

In [84]:
n_grams = 2 # Numero de palavras anteriores para prever a próxima
# Criação de conjuntos de palavras de tamanho n_grams
sets_of_k_words = [ ' '.join(corpus_words[i:i+n_grams]) for i, _ in enumerate(corpus_words[:-n_grams]) ]
sets_count = len(list(set(sets_of_k_words)))

print(sets_of_k_words[:10])

['dom casmurro', 'casmurro capítulo', 'capítulo primeiro', 'primeiro do', 'do título', 'título uma', 'uma noite', 'noite destas', 'destas vindo', 'vindo da']


In [85]:
# Criação de uma matriz esparsa para armazenar a contagem de palavras
next_after_k_words_matrix = dok_matrix((sets_count, distinct_words_count))

# Criação de um dicionário para mapear os conjuntos de palavras para índices
distinct_sets_of_k_words = list(set(sets_of_k_words))
k_words_idx_dict = {word: i for i, word in enumerate(distinct_sets_of_k_words)}

# Preenchimento da matriz com as contagens
for i, word in enumerate(sets_of_k_words[:-n_grams]):
    word_sequence_idx = k_words_idx_dict[word]
    next_word_idx = word_idx_dict[corpus_words[i+n_grams]]
    next_after_k_words_matrix[word_sequence_idx, next_word_idx] +=1
    
print("Quantidade de vezes que os bigramas ocorrem juntos")
print(next_after_k_words_matrix[:10])

Quantidade de vezes que os bigramas ocorrem juntos
  (7, 7136)	1.0
  (2, 1264)	1.0
  (2, 6410)	1.0
  (5, 3862)	1.0
  (8, 3029)	1.0
  (6, 3862)	1.0
  (2, 4027)	1.0
  (2, 3281)	1.0
  (2, 6168)	1.0
  (2, 6799)	1.0
  (2, 481)	1.0
  (2, 3097)	1.0
  (0, 6190)	1.0
  (2, 7818)	1.0
  (1, 5060)	1.0
  (9, 8763)	1.0
  (3, 2634)	1.0
  (4, 6441)	1.0


In [86]:
def weighted_choice(objects, weights):
    """ 
    Função que retorna um elemento aleatório da sequência de 'objetos',
    a probabilidade dos objetos é ponderada de acordo com a porcentagem.
    """
    weights = np.array(weights, dtype=np.float64)
    sum_of_weights = weights.sum()
    # standardization:
    np.multiply(weights, 1 / sum_of_weights, weights)
    weights = weights.cumsum()
    x = random()
    for i in range(len(weights)):
        if x < weights[i]:
            return objects[i]

In [87]:
def sample_next_word_after_sequence(word_sequence, alpha = 0):
    """
    Função que retorna a próxima palavra após a sequência de palavras
    """
    next_word_vector = next_after_k_words_matrix[k_words_idx_dict[word_sequence]] + alpha
    likelihoods = next_word_vector/next_word_vector.sum()
    
    return weighted_choice(distinct_words, likelihoods.toarray())
    
def markov_chain(initial_text, chain_length=30):
    """
    Função que gera um texto de acordo com o texto inicial
    """
    current_words = initial_text.split(' ')
    sentence = initial_text

    for _ in range(chain_length):
        sentence+=' '
        next_word = sample_next_word_after_sequence(' '.join(current_words))
        sentence+=next_word
        current_words = current_words[1:]+[next_word]
    return sentence

In [88]:
markov_chain('dom casmurro')

'dom casmurro domingo vou jantar com você basta isto? devia bastar e depois de jantar estávamos ainda à mesa do jantar cozidas nunca comi os ninhos delas continuou mas devem ser todas'

In [90]:
markov_chain('capitu era')

'capitu era parecida com o dedo na boca de capitu você não vier logo e parece que a das crianças foi que à primeira suspeita e a família ou com a vida'

In [91]:
markov_chain('eu sou')

'eu sou acaso um deles conquanto a prova não provava todas as ilhas de todos mas se foi ele mesmo que acender em mim com tanto prazer que eu não saiba nem'